In [1]:
import matplotlib
matplotlib.use("nbagg")

import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D

from utility import *
from models import *

from os import listdir
from os.path import isfile, join
import numpy as np
import math

import torch.optim as optim

# Define from where to load the files 

In [2]:
polycube_path = "/Users/davidcleres/DeepShape/Polycubing-Automated/Generated-Cars/"
polycube_files = [f for f in listdir(polycube_path) if isfile(join(polycube_path, f))]

voxelized_mesh_path = "/Users/davidcleres/DeepShape/Polycubing-Automated/voxelizedMeshes/"
voxelized_mesh_files = [f for f in listdir(voxelized_mesh_path) if isfile(join(voxelized_mesh_path, f))]

voxelizedFiles = []
polycubedFiles = []

for f in voxelized_mesh_files: 
    if f[-13:] == "voxelized.txt":
        voxelizedFiles = np.hstack((voxelizedFiles, f))
    
for f in polycube_files:
    if f[-14:] == "finalCubes.txt":
        polycubedFiles = np.hstack((polycubedFiles, f))

# Load the data

In [3]:
grid_size = 32
voxelized_train_input, polycube_target=loadData(grid_size, polycube_path, voxelized_mesh_path, voxelizedFiles, polycubedFiles, loadFromScratch=False)

# Build a training and validation set 

In [4]:
batch_size = 10 
validation_size=50
preprocessed_input_train, preprocessed_input_validation, preprocessed_input_train_target, preprocessed_input_validation_target = preprocessing_train(voxelized_train_input, polycube_target, validation_size, False, False)

preprocessed_input_train = torch.from_numpy(preprocessed_input_train)
preprocessed_input_validation = torch.from_numpy(preprocessed_input_validation)
preprocessed_input_train_target = torch.from_numpy(preprocessed_input_train_target)
preprocessed_input_validation_target = torch.from_numpy(preprocessed_input_validation_target)

Ntrain = len(preprocessed_input_train[:,0,0,0,0]) 
Nvalidation = len(preprocessed_input_validation[:,0,0,0,0])
image_size = 32

train_input = np.array(preprocessed_input_train.view(Ntrain, 1,image_size, image_size, image_size))
validation_input = np.array(preprocessed_input_validation.view(Nvalidation, 1,image_size, image_size, image_size))

labels_train = np.array(preprocessed_input_train_target.view(Ntrain, 1, image_size, image_size, image_size))
labels_validation = np.array(preprocessed_input_validation_target.view(Nvalidation, 1,image_size, image_size, image_size))

# Create the target 
Find the middle of the cube and the distance to the x, y and z boundaries

In [5]:
labels_train_cube_coords = np.zeros((len(train_input[:,0,0,0,0]), 1, 9))
for i in range (len(train_input[:,0,0,0,0])):
    #solution 1 - the loss is calculed based on the 9 labels 
    delta_x_left, delta_x_right, center_x, delta_y_left, delta_y_right, center_y, delta_z_left, delta_z_right, center_z = find_center_and_delta(labels_train[i, 0, :, :, :], grid_size=32)
    labels_train_cube_coords[i,0,0] = delta_x_left
    labels_train_cube_coords[i,0,1] = delta_x_right
    labels_train_cube_coords[i,0,2] = center_x
    labels_train_cube_coords[i,0,3] = delta_y_left
    labels_train_cube_coords[i,0,4] = delta_y_right
    labels_train_cube_coords[i,0,5] = center_y
    labels_train_cube_coords[i,0,6] = delta_z_left
    labels_train_cube_coords[i,0,7] = delta_z_right
    labels_train_cube_coords[i,0,8] = center_z
    
labels_validation_cube_coords = np.zeros((len(labels_validation[:,0,0,0,0]), 1, 9))
for i in range (len((validation_input[:,0,0,0,0]))):
    #solution 1 - the loss is calculed based on the 9 labels 
    delta_x_left, delta_x_right, center_x, delta_y_left, delta_y_right, center_y, delta_z_left, delta_z_right, center_z = find_center_and_delta(labels_train[i, 0, :, :, :], grid_size=32)
    labels_validation_cube_coords[i, 0,0] = delta_x_left
    labels_validation_cube_coords[i, 0,1] = delta_x_right
    labels_validation_cube_coords[i, 0,2] = center_x
    labels_validation_cube_coords[i, 0,3] = delta_y_left
    labels_validation_cube_coords[i, 0,4] = delta_y_right
    labels_validation_cube_coords[i, 0,5] = center_y
    labels_validation_cube_coords[i, 0,6] = delta_z_left
    labels_validation_cube_coords[i, 0,7] = delta_z_right
    labels_validation_cube_coords[i, 0,8] = center_z
    
    
labels_validation_cube_coords = torch.from_numpy(labels_validation_cube_coords)
labels_train_cube_coords = torch.from_numpy(labels_train_cube_coords)
train_input = torch.from_numpy(train_input)
validation_input = torch.from_numpy(validation_input)

print('train_input', train_input.shape)
print('labels_train_cube_coords', labels_train_cube_coords.shape)

print('validation_input', validation_input.shape)
print('labels_validation_cube_coords', labels_validation_cube_coords.shape)

    
    #solution 2 - the loss is calculated from the cube generated thanks to the learned coordinates 
    #output = build_cube(delta_x_left, delta_x_right, center_x, delta_y_left, delta_y_right, center_y, delta_z_left, delta_z_right, center_z, grid_size)
    

train_input torch.Size([550, 1, 32, 32, 32])
labels_train_cube_coords torch.Size([550, 1, 9])
validation_input torch.Size([50, 1, 32, 32, 32])
labels_validation_cube_coords torch.Size([50, 1, 9])


In [ ]:
# Train network 
#criterion = nn.BCELoss()
#criterion = nn.CrossEntropyLoss()
#criterion = nn.PoissonNLLLoss()
#criterion = nn.BCEWithLogitsLoss()
#criterion = nn.SmoothL1Loss()
criterion = nn.MSELoss()

train_input = Variable(train_input).float()
validation_input = Variable(validation_input).float()

if isinstance(criterion, nn.CrossEntropyLoss):
    train_target = Variable(labels_train_cube_coords)  # keep long tensors
    validation_target = Variable(labels_validation_cube_coords, 
    uires_grad=False) # convert to float
    Noutputs = 18
    
elif isinstance(criterion, nn.NLLLoss):
    train_target = Variable(labels_train_cube_coords)  # keep long tensors
    validation_target = Variable(labels_validation_cube_coords, requires_grad=False) # convert to float
    Noutputs = 18
    
else:
    train_target = Variable(labels_train_cube_coords, requires_grad=False).float() # convert to float
    validation_target = Variable(labels_validation_cube_coords, requires_grad=False).float() # convert to float
    Noutputs = 9

Nbatches = int(math.ceil(Ntrain/batch_size))
Nepochs = 500
#seeds = list(range(15)) #Test 15 different seeds but always the seeds from 0 to 15 so that the weights are always initialized in a reproducible way
#Nrep = len(seeds)
Nrep = 1

train_errors = torch.Tensor(Nrep, Nepochs).zero_()
test_errors = torch.Tensor(Nrep, Nepochs).zero_()
validation_errors = torch.Tensor(Nrep, Nepochs).zero_()
ep_loss = torch.Tensor(Nrep, Nepochs).zero_()

for i_rep in range(Nrep):    
    #print('Repetition', seeds[i_rep])
    #torch.manual_seed(seeds[i_rep])
    
    #model = conv2DNet_1(Nchannels, Nsamples_100, Noutputs) #from litterature EEG-Net coorected
    model = conv2DNet_2(Noutputs)  #from Temporal - Spatial; 4 Filters Model - Best performing model with accuracy 0.83 in average on the validation set
    #model = conv2DNet_3(Noutputs) #from Temporal - Spatial; 64 Filters Model
    #model = conv2DNet_4(Noutputs) #from Temporal - Spatial; 128 Filters Model
    
    #optimizer = optim.SGD(model.parameters(), lr=1e-3, momentum=0.50)
    optimizer = optim.Adam(model.parameters())
    #optimizer = optim.Adagrad(model.parameters())
    #optimizer = optim.Adamax(model.parameters())
    #optimizer = optim.ASGD(model.parameters())
    #optimizer = optim.RMSprop(model.parameters())
    #optimizer = optim.Rprop(model.parameters())
    
    #scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=10, verbose=True) #Reduces the learning rate if it did not decreased by more than 10^-4 in 10 steps

    for i_ep in range(Nepochs):
        for b_start in range(0, Ntrain, batch_size):
            bsize_eff = batch_size - max(0, b_start+batch_size-Ntrain)  # boundary case
            model.train()
            model.zero_grad()
            output = model(train_input.narrow(0, b_start, bsize_eff))
            if isinstance(criterion, nn.CrossEntropyLoss) or isinstance(criterion, nn.NLLLoss):
                batch_loss = criterion(output, train_target.narrow(0, b_start, bsize_eff))
            else:
                batch_loss = criterion(output.view(bsize_eff*Noutputs), train_target.narrow(0, b_start, bsize_eff))
            ep_loss[i_rep, i_ep] += batch_loss.data[0]
            batch_loss.backward()
            optimizer.step()
        
        #scheduler.step(ep_loss[i_rep, i_ep])
        
        nb_train_errs = compute_nb_errors_delta(model, train_input, train_target, batch_size, criterion)
        nb_validation_errs = compute_nb_errors_delta(model, validation_input, validation_target, batch_size, criterion)
        
        print("Epoch Number : ", i_ep)
        print("\t Training accuracy: ", (100*(Ntrain*Noutputs-nb_train_errs)/(Ntrain*Noutputs)))
        print("\t Validation accuracy ",(100*(Nvalidation*Noutputs-nb_validation_errs)/(Nvalidation*Noutputs)))
        
        print("\t Epoch Loss ", ep_loss[i_rep, i_ep])
        
        train_errors[i_rep, i_ep] = nb_train_errs
        validation_errors[i_rep, i_ep] = nb_validation_errs

Epoch Number :  0
	 Training accuracy:  98.28282828282828
	 Validation accuracy  80.44444444444444
	 Epoch Loss  55376.265625
Epoch Number :  1
	 Training accuracy:  98.26262626262626
	 Validation accuracy  80.0
	 Epoch Loss  12929.423828125
Epoch Number :  2
	 Training accuracy:  98.18181818181819
	 Validation accuracy  80.0
	 Epoch Loss  6713.62939453125
Epoch Number :  3
	 Training accuracy:  98.18181818181819
	 Validation accuracy  80.0
	 Epoch Loss  24598.0078125
Epoch Number :  4
	 Training accuracy:  98.22222222222223
	 Validation accuracy  80.88888888888889
	 Epoch Loss  21536.29296875
Epoch Number :  5
	 Training accuracy:  98.28282828282828
	 Validation accuracy  81.11111111111111
	 Epoch Loss  9003.287109375
Epoch Number :  6
	 Training accuracy:  98.32323232323232
	 Validation accuracy  80.88888888888889
	 Epoch Loss  9528.4267578125
Epoch Number :  7
	 Training accuracy:  98.26262626262626
	 Validation accuracy  80.44444444444444
	 Epoch Loss  10330.45703125
Epoch Number :

Epoch Number :  64
	 Training accuracy:  98.56565656565657
	 Validation accuracy  83.11111111111111
	 Epoch Loss  394.8517761230469
Epoch Number :  65
	 Training accuracy:  98.62626262626263
	 Validation accuracy  83.11111111111111
	 Epoch Loss  422.125732421875
Epoch Number :  66
	 Training accuracy:  98.68686868686869
	 Validation accuracy  83.77777777777777
	 Epoch Loss  450.8711242675781
Epoch Number :  67
	 Training accuracy:  98.5050505050505
	 Validation accuracy  82.44444444444444
	 Epoch Loss  629.8591918945312
Epoch Number :  68
	 Training accuracy:  98.38383838383838
	 Validation accuracy  82.22222222222223
	 Epoch Loss  889.16455078125
Epoch Number :  69
	 Training accuracy:  98.36363636363636
	 Validation accuracy  81.33333333333333
	 Epoch Loss  1089.98779296875
Epoch Number :  70
	 Training accuracy:  98.48484848484848
	 Validation accuracy  82.22222222222223
	 Epoch Loss  1483.556640625
Epoch Number :  71
	 Training accuracy:  98.42424242424242
	 Validation accuracy  82

Epoch Number :  127
	 Training accuracy:  99.71717171717172
	 Validation accuracy  86.22222222222223
	 Epoch Loss  28.993694305419922
Epoch Number :  128
	 Training accuracy:  99.75757575757575
	 Validation accuracy  84.88888888888889
	 Epoch Loss  27.320964813232422
Epoch Number :  129
	 Training accuracy:  99.47474747474747
	 Validation accuracy  84.66666666666667
	 Epoch Loss  29.106718063354492
Epoch Number :  130
	 Training accuracy:  99.51515151515152
	 Validation accuracy  83.77777777777777
	 Epoch Loss  34.087425231933594
Epoch Number :  131
	 Training accuracy:  99.55555555555556
	 Validation accuracy  83.33333333333333
	 Epoch Loss  35.78203201293945
Epoch Number :  132
	 Training accuracy:  99.31313131313131
	 Validation accuracy  84.22222222222223
	 Epoch Loss  39.318443298339844
Epoch Number :  133
	 Training accuracy:  99.05050505050505
	 Validation accuracy  84.66666666666667
	 Epoch Loss  39.98631286621094
Epoch Number :  134
	 Training accuracy:  99.05050505050505
	 Va

Epoch Number :  190
	 Training accuracy:  99.85858585858585
	 Validation accuracy  85.77777777777777
	 Epoch Loss  25.74904441833496
Epoch Number :  191
	 Training accuracy:  99.1919191919192
	 Validation accuracy  83.55555555555556
	 Epoch Loss  23.24734878540039
Epoch Number :  192
	 Training accuracy:  99.83838383838383
	 Validation accuracy  84.22222222222223
	 Epoch Loss  25.879850387573242
Epoch Number :  193
	 Training accuracy:  99.47474747474747
	 Validation accuracy  83.55555555555556
	 Epoch Loss  26.992956161499023
Epoch Number :  194
	 Training accuracy:  99.27272727272727
	 Validation accuracy  85.55555555555556
	 Epoch Loss  33.75056076049805
Epoch Number :  195
	 Training accuracy:  99.5959595959596
	 Validation accuracy  85.11111111111111
	 Epoch Loss  30.32574462890625
Epoch Number :  196
	 Training accuracy:  99.25252525252525
	 Validation accuracy  82.66666666666667
	 Epoch Loss  26.401893615722656
Epoch Number :  197
	 Training accuracy:  99.23232323232324
	 Valida

Epoch Number :  252
	 Training accuracy:  99.37373737373737
	 Validation accuracy  84.44444444444444
	 Epoch Loss  29.092912673950195
Epoch Number :  253
	 Training accuracy:  99.63636363636364
	 Validation accuracy  84.44444444444444
	 Epoch Loss  23.389135360717773
Epoch Number :  254
	 Training accuracy:  99.39393939393939
	 Validation accuracy  85.11111111111111
	 Epoch Loss  24.260356903076172
Epoch Number :  255
	 Training accuracy:  99.27272727272727
	 Validation accuracy  84.44444444444444
	 Epoch Loss  37.133365631103516
Epoch Number :  256
	 Training accuracy:  99.5959595959596
	 Validation accuracy  85.55555555555556
	 Epoch Loss  42.649234771728516
Epoch Number :  257
	 Training accuracy:  99.47474747474747
	 Validation accuracy  84.44444444444444
	 Epoch Loss  34.64773941040039
Epoch Number :  258
	 Training accuracy:  99.4949494949495
	 Validation accuracy  87.11111111111111
	 Epoch Loss  25.171707153320312
Epoch Number :  259
	 Training accuracy:  99.31313131313131
	 Val

Epoch Number :  315
	 Training accuracy:  99.39393939393939
	 Validation accuracy  84.66666666666667
	 Epoch Loss  28.30374526977539
Epoch Number :  316
	 Training accuracy:  99.55555555555556
	 Validation accuracy  84.22222222222223
	 Epoch Loss  30.365013122558594
Epoch Number :  317
	 Training accuracy:  99.35353535353535
	 Validation accuracy  83.77777777777777
	 Epoch Loss  30.435516357421875
Epoch Number :  318
	 Training accuracy:  99.41414141414141
	 Validation accuracy  84.22222222222223
	 Epoch Loss  21.596311569213867
Epoch Number :  319
	 Training accuracy:  99.57575757575758
	 Validation accuracy  84.44444444444444
	 Epoch Loss  26.301040649414062
Epoch Number :  320
	 Training accuracy:  98.96969696969697
	 Validation accuracy  86.22222222222223
	 Epoch Loss  27.75873565673828
Epoch Number :  321
	 Training accuracy:  98.84848484848484
	 Validation accuracy  85.55555555555556
	 Epoch Loss  31.012319564819336
Epoch Number :  322
	 Training accuracy:  99.37373737373737
	 Va

Epoch Number :  377
	 Training accuracy:  99.23232323232324
	 Validation accuracy  84.88888888888889
	 Epoch Loss  50.03337097167969
Epoch Number :  378
	 Training accuracy:  99.15151515151516
	 Validation accuracy  86.88888888888889
	 Epoch Loss  62.6310920715332
Epoch Number :  379
	 Training accuracy:  99.17171717171718
	 Validation accuracy  84.66666666666667
	 Epoch Loss  51.94892120361328
Epoch Number :  380
	 Training accuracy:  99.11111111111111
	 Validation accuracy  84.88888888888889
	 Epoch Loss  58.14137268066406
Epoch Number :  381
	 Training accuracy:  99.05050505050505
	 Validation accuracy  82.66666666666667
	 Epoch Loss  87.2336654663086
Epoch Number :  382
	 Training accuracy:  99.01010101010101
	 Validation accuracy  84.0
	 Epoch Loss  41.78303909301758
Epoch Number :  383
	 Training accuracy:  99.11111111111111
	 Validation accuracy  82.88888888888889
	 Epoch Loss  34.62181854248047
Epoch Number :  384
	 Training accuracy:  99.17171717171718
	 Validation accuracy  8

Epoch Number :  440
	 Training accuracy:  99.79797979797979
	 Validation accuracy  87.11111111111111
	 Epoch Loss  15.485883712768555
Epoch Number :  441
	 Training accuracy:  99.63636363636364
	 Validation accuracy  88.88888888888889
	 Epoch Loss  11.840843200683594
Epoch Number :  442
	 Training accuracy:  99.31313131313131
	 Validation accuracy  86.0
	 Epoch Loss  19.921611785888672


# NETWORK SUMMARY & RESULTS

In [ ]:
train_accuracy = 100*(Ntrain-np.array(train_errors))/Ntrain
val_accuracy = 100*(Nvalidation-np.array(validation_errors))/Nvalidation
np.save('training_accuracy_Adamax', train_accuracy)
np.save('validation_accuracy_Adamax', val_accuracy)

stddev_train_errors = np.std(train_accuracy, axis=0)
stddev_val_errors = np.std(val_accuracy, axis=0)

mean_train_errors = np.mean(train_accuracy, axis=0)
mean_val_errors = np.mean(val_accuracy, axis=0)

epoch = list(range(50))
plt.plot(epoch, mean_train_errors)
plt.plot(epoch, mean_val_errors)
plt.fill_between(epoch, mean_train_errors+stddev_train_errors, mean_train_errors-stddev_train_errors, alpha=0.5)
plt.fill_between(epoch, mean_val_errors+stddev_val_errors, mean_val_errors-stddev_val_errors, alpha=0.5)
plt.xlabel('Number of epochs')
plt.ylabel('Accuracy in %')
plt.legend(['train', 'validation', 'test'])

print("Training accuracy {:4.3g}%+-{}".format(mean_train_errors[-1], stddev_train_errors[-1]))
print("Validation accuracy {:4.3g}%+-{}".format(mean_val_errors[-1], stddev_val_errors[-1]))

In [ ]:
output.round()

In [ ]:
train_target.narrow(0, b_start, bsize_eff)